In [299]:
import pickle
from collections import defaultdict
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA

## Load Joint History Data

In [300]:
def load_pickle(file_name):
  with open(f'{file_name}.pickle', 'rb') as f:
      kpts = pickle.load(f)
  
  return kpts

## Transform Data into Joint Deltas

In [301]:
def compute_deltas(joint_history: dict) -> dict:
  joint_deltas = defaultdict(list)

  for joint in joint_history:
    for i, angle in enumerate(joint_history[joint]):
      if i == 0: continue

      joint_deltas[joint].append(angle - joint_history[joint][i-1])
  
  return joint_deltas


def extract_features(joint_history: dict, discriminative_joints: set) -> dict:
  window_size = 64
  joint_deltas = compute_deltas({
    key: joint_history[key] for key in discriminative_joints})
  
  joint_deltas["knees"] = (np.array(joint_history["knee_right"]) + np.array(joint_history["knee_left"])) / 2
  # joint_deltas["elbow"] = (np.array(joint_history["elbow_right"]) + np.array(joint_history["elbow_left"])) / 2

  del joint_deltas["knee_left"]
  del joint_deltas["knee_right"]
  # del joint_deltas["elbow_right"]
  # del joint_deltas["elbow_left"]
  
  joint_features = defaultdict(list)

  for joint in joint_deltas:
    i = 0
    j = i + window_size

    while j < len(joint_deltas[joint]):
      joint_features[joint].append(np.array(joint_deltas[joint])[i:j])
      i += 1
      j += 1

  return joint_features

In [302]:
joints_history_squat = load_pickle("squat_static_ktps_joints")
joints_history_pullup = load_pickle("joints_history_pullup")

discriminative_joints = set(
    ["knee_right", "knee_left"])#, "elbow_left", "elbow_right"])

joint_features_squat = extract_features(joints_history_squat, discriminative_joints)
joint_features_pullup = extract_features(joints_history_pullup, discriminative_joints)

print(joint_features_pullup)


defaultdict(<class 'list'>, {'knees': [array([158.56712062, 157.97875557, 157.28707217, 156.81726754,
       156.68764837, 156.42760062, 155.79938883, 155.80567794,
       155.00638856, 154.76481494, 154.60646746, 154.93830508,
       155.18904451, 155.12490723, 155.22101424, 155.40642821,
       155.58767777, 155.22329623, 155.23778109, 152.45901093,
       149.12493611, 146.26406763, 144.31404487, 143.83606872,
       145.06400265, 145.29534884, 146.85229149, 150.47337466,
       152.6684034 , 152.37554555, 152.72108526, 152.26839252,
       150.08956545, 150.25960609, 150.57533553, 150.68233184,
       150.47941198, 150.90508673, 150.6837691 , 150.66672682,
       150.95753705, 151.09080922, 151.42983542, 151.42934915,
       151.48072781, 151.80393946, 151.95994937, 151.90611924,
       152.46572319, 152.52654128, 152.77073303, 152.77059094,
       152.46526858, 152.23798882, 152.59054312, 152.67909864,
       152.33982911, 152.7493338 , 152.82824425, 152.54385393,
       152.80873

## Build Dataset

In [303]:
def build_dataset(joint_features: dict, discriminative_joints: set, label: int, dataset: list):
  for i in range(len(joint_features[list(discriminative_joints)[0]])):  # Assuming all joints have the same number of features
      combined_features = []
      
      for joint in discriminative_joints:
          combined_features.extend(joint_features[joint][i])

      dataset.append((combined_features, label))
  
  return dataset

In [305]:
dataset = build_dataset(joint_features_pullup, ["knees"], 0, [])#, "elbow"], 0, [])
dataset = build_dataset(joint_features_squat, ["knees"], 1, dataset)#, "elbow"], 1, dataset)

np.array(dataset[0][0]).shape

(64,)

## Train and Evaluate with SVC

In [306]:
X, y = [data[0] for data in dataset], [data[1] for data in dataset]
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.10, random_state=42)
y_train = y_train.reshape(-1, 1)

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)

/Users/patricktourniaire/Library/Caches/pypoetry/virtualenvs/workout-verify-ltG9Ybba-py3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [307]:
X_test.shape

(96, 64)

In [308]:
y_pred = clf.predict(X_test)
f1_score(y_test, y_pred, average='macro')

1.0

In [309]:
import pickle

# save
with open('sliding_window_classifier.pkl','wb') as f:
    pickle.dump(clf,f)